# 階層的インデックス

In [1]:
import pandas as pd
import numpy as np

## 複数インデックスSeries
### 悪い例
単純なタプルをインデックスに使用した悪い例。

In [2]:
index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)
pop

(California, 2000)    33871648
(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
(Texas, 2010)         25145561
dtype: int64

In [3]:
pop[('California', 2010):('Texas', 2000)]

(California, 2010)    37253956
(New York, 2000)      18976457
(New York, 2010)      19378102
(Texas, 2000)         20851820
dtype: int64

In [4]:
pop[[i for i in pop.index if i[1] == 2010]]

(California, 2010)    37253956
(New York, 2010)      19378102
(Texas, 2010)         25145561
dtype: int64

### よりよい例： PandasのMultiIndex
Pandasの`MultiIndex`型がやりたい操作の方法を与える。以下の様に、タプルからマルチインデックスを作成することが出来る。

In [6]:
index = pd.MultiIndex.from_tuples(index)
index

MultiIndex(levels=[['California', 'New York', 'Texas'], [2000, 2010]],
           labels=[[0, 0, 1, 1, 2, 2], [0, 1, 0, 1, 0, 1]])

ここでは、それぞれ州の名前と年のインデックスを示すlevelsと、これらのlevelsにエンコードされたデータポイントに対するlabelsを含んでいる。

もし、この`MultiIndex`でSeriesを再インデックスすれば、データを階層的なものにすることが出来る。

In [7]:
pop = pop.reindex(index)
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

ここで、始めの2つの列が`Series`の複数インデックスの値を示しており、3列目がそのデータを示す。1行目の空白となっている部分は、その上の部分と同じエントリーであることを示している。

2010年の全てのデータを取得したい場合には、Pandasのスライス表現を単に使用するだけである。

In [8]:
pop[:, 2010]

California    37253956
New York      19378102
Texas         25145561
dtype: int64

### MultiIndexでの追加の次元
`DataFrame`を使用すれば、似たようなことが出来るかもしれない。`unstack()`メソッドを利用すれば、複数インデックスの`Series`を`DataFrame`に変換できる。

In [9]:
pop_df = pop.unstack()
pop_df

,2000,2010
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


`stack()`を利用すれば反対のことが出来る。

In [10]:
pop_df.stack()

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

だったらなぜわざわざMultiIndexなんて使用するのか。それは2次元だけでなく、3次元でも、それ以上でも表現できるからである。

`MultiIndex`を使って`DataFrame`に新たな列を追加する。

In [11]:
pop_df = pd.DataFrame({'total': pop,
                       'under18': [9267089, 9284094,
                                   4687374, 4318033,
                                   5906301, 6879014]})
pop_df

total  under18
California 2000  33871648  9267089
           2010  37253956  9284094
New York   2000  18976457  4687374
           2010  19378102  4318033
Texas      2000  20851820  5906301
           2010  25145561  6879014

加えて、全てのufuncsなども使える。

In [14]:
f_u18 = pop_df['under18'] / pop_df['total']
f_u18

California  2000    0.273594
            2010    0.249211
New York    2000    0.247010
            2010    0.222831
Texas       2000    0.283251
            2010    0.273568
dtype: float64

## MultiIndex 作成のメソッド
一番簡単なのは2つ以上のインデックスのarrayをコンストラクタに渡すことである。

In [15]:
df = pd.DataFrame(np.random.rand(4, 2),
                  index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                  columns=['data1', 'data2'])
df

data1     data2
a 1  0.095546  0.440176
  2  0.975008  0.961358
b 1  0.552259  0.893294
  2  0.382726  0.893927

同様に、適切なタプルをキーとして使用する辞書を渡した場合にも、Pandasは自動的にこれを理解し、`MultiIndex`をデフォルトで使用する。

In [16]:
data = {('California', 2000): 33871648,
        ('California', 2010): 37253956,
        ('Texas', 2000): 20851820,
        ('Texas', 2010): 25145561,
        ('New York', 2000): 18976457,
        ('New York', 2010): 19378102}
pd.Series(data)

California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
New York    2000    18976457
            2010    19378102
dtype: int64

### 厳密なMultiIndexのコンストラクタ

In [17]:
pd.MultiIndex.from_arrays([['a', 'a', 'b', 'b'], [1, 2, 1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [18]:
pd.MultiIndex.from_tuples([('a', 1), ('a', 2), ('b', 1), ('b', 2)])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [19]:
pd.MultiIndex.from_product([['a', 'b'], [1, 2]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

In [20]:
pd.MultiIndex(levels=[['a', 'b'], [1, 2]],
              labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

MultiIndex(levels=[['a', 'b'], [1, 2]],
           labels=[[0, 0, 1, 1], [0, 1, 0, 1]])

### MultiIndexのlevel名

In [21]:
pop

California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [22]:
pop.index.names = ['state', 'year']
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### 列に対するMultiIndex
`DataFrame`は列も行も対称的であるため、行が複数のレベルのインデックスを持てるように、列も複数のレベルを持つことが出来る。

In [23]:
# hierarchical indices and columns
index = pd.MultiIndex.from_product([[2013, 2014], [1, 2]],
                                   names=['year', 'visit'])
columns = pd.MultiIndex.from_product([['Bob', 'Guido', 'Sue'], ['HR', 'Temp']],
                                     names=['subject', 'type'])

# mock some data
data = np.round(np.random.randn(4, 6), 1)
data[:, ::2] *= 10
data += 37

# create the DataFrame
health_data = pd.DataFrame(data, index=index, columns=columns)
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      23.0  37.2  31.0  37.0  40.0  37.7
     2      39.0  35.7  33.0  36.4  32.0  38.3
2014 1      32.0  37.0  35.0  36.6  26.0  37.1
     2      28.0  39.1  57.0  36.8  23.0  36.7

In [24]:
health_data['Guido']

type          HR  Temp
year visit            
2013 1      31.0  37.0
     2      33.0  36.4
2014 1      35.0  36.6
     2      57.0  36.8

## MultiIndexのインデックスとスライス
### Seriesのインデックス

In [25]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [26]:
pop['California', 2000]

33871648

In [27]:
pop['California']

year
2000    33871648
2010    37253956
dtype: int64

In [28]:
pop.loc['California':'New York']

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
dtype: int64

In [29]:
pop[:, 2000]

state
California    33871648
New York      18976457
Texas         20851820
dtype: int64

In [30]:
pop[pop > 22000000]

state       year
California  2000    33871648
            2010    37253956
Texas       2010    25145561
dtype: int64

In [31]:
pop[['California', 'Texas']]

state       year
California  2000    33871648
            2010    37253956
Texas       2000    20851820
            2010    25145561
dtype: int64

### Dataframeのインデックス

In [32]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      23.0  37.2  31.0  37.0  40.0  37.7
     2      39.0  35.7  33.0  36.4  32.0  38.3
2014 1      32.0  37.0  35.0  36.6  26.0  37.1
     2      28.0  39.1  57.0  36.8  23.0  36.7

In [33]:
health_data['Guido', 'HR']

year  visit
2013  1        31.0
      2        33.0
2014  1        35.0
      2        57.0
Name: (Guido, HR), dtype: float64

In [34]:
health_data.iloc[:2, :2]

subject      Bob      
type          HR  Temp
year visit            
2013 1      23.0  37.2
     2      39.0  35.7

In [35]:
health_data.loc[:, ('Bob', 'HR')]

year  visit
2013  1        23.0
      2        39.0
2014  1        32.0
      2        28.0
Name: (Bob, HR), dtype: float64

In [36]:
health_data.loc[(:, 1), (:, 'HR')]

SyntaxError: invalid syntax (<ipython-input-36-fb34fa30ac09>, line 1)

In [37]:
idx = pd.IndexSlice
health_data.loc[idx[:, 1], idx[:, 'HR']]

,subject,Bob,Guido,Sue
,type,HR,HR,HR
year,visit,,,
2013,1,23.0,31.0,40.0
2014,1,32.0,35.0,26.0


# Multiインデックスの再配置
多くのMultiIndexにおいて、インデックスがソートされていないとスライス操作は失敗する。

In [38]:
index = pd.MultiIndex.from_product([['a', 'c', 'b'], [1, 2]])
data = pd.Series(np.random.rand(6), index=index)
data.index.names = ['char', 'int']
data

char  int
a     1      0.914202
      2      0.805991
c     1      0.963710
      2      0.931221
b     1      0.252676
      2      0.677166
dtype: float64

In [39]:
try:
    data['a':'b']
except KeyError as e:
    print(type(e))
    print(e)

<class 'pandas.errors.UnsortedIndexError'>
'Key length (1) was greater than MultiIndex lexsort depth (0)'


Pandasはいくつかのソートを用意している。

最もシンプルなのは`sort_index()`である。

In [40]:
data = data.sort_index()
data

char  int
a     1      0.914202
      2      0.805991
b     1      0.252676
      2      0.677166
c     1      0.963710
      2      0.931221
dtype: float64

このソートの方法によって、スライスが思ったように動作するようになる。

### インデックスのスタックとアンスタック
Multi-indexから2次元の表現に変換することができ、オプションとしてlevelを利用することが出来る。

In [41]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [42]:
pop.unstack(level=0)

state,California,New York,Texas
year,,,
2000,33871648,18976457,20851820
2010,37253956,19378102,25145561


In [43]:
pop.unstack(level=1)

year,2000,2010
state,,
California,33871648,37253956
New York,18976457,19378102
Texas,20851820,25145561


`unstack()`と反対の操作を`stack()`で利用できる。

In [44]:
pop.unstack().stack()

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

### indexのセットとリセット
もうひとつの再配置はインデックスを列にすることである。これは`reset_index`メソッドで行える。

In [45]:
pop

state       year
California  2000    33871648
            2010    37253956
New York    2000    18976457
            2010    19378102
Texas       2000    20851820
            2010    25145561
dtype: int64

In [46]:
pop_flat = pop.reset_index(name='population')
pop_flat

,state,year,population
0,California,2000,33871648
1,California,2010,37253956
2,New York,2000,18976457
3,New York,2010,19378102
4,Texas,2000,20851820
5,Texas,2010,25145561


現実の世界では上記のような生の状態のデータをよく見る。このような列のデータから`MultiIndex`を作成できれば便利である。これは`set_index`を使って実現できる。

In [47]:
pop_flat.set_index(['state', 'year'])

population
state      year            
California 2000    33871648
           2010    37253956
New York   2000    18976457
           2010    19378102
Texas      2000    20851820
           2010    25145561

## マルチインデックスでのデータアグリゲーション

In [48]:
health_data

subject      Bob       Guido         Sue      
type          HR  Temp    HR  Temp    HR  Temp
year visit                                    
2013 1      23.0  37.2  31.0  37.0  40.0  37.7
     2      39.0  35.7  33.0  36.4  32.0  38.3
2014 1      32.0  37.0  35.0  36.6  26.0  37.1
     2      28.0  39.1  57.0  36.8  23.0  36.7

In [49]:
data_mean = health_data.mean(level='year')
data_mean

subject   Bob        Guido         Sue      
type       HR   Temp    HR  Temp    HR  Temp
year                                        
2013     31.0  36.45  32.0  36.7  36.0  38.0
2014     30.0  38.05  46.0  36.7  24.5  36.9

In [50]:
data_mean.mean(axis=1, level='type')

type,HR,Temp
year,,
2013,33.0,37.050000
2014,33.5,37.216667
